In [ ]:
import cv2
import numpy as np
import tkinter as tk
from tkinter import Label, Button, Frame
from PIL import Image, ImageTk
from ttkthemes import ThemedTk

# Load cascades and model
face_cascade = cv2.CascadeClassifier(r'C:\Users\HP\Desktop\Machine learning\haarcascade_frontface.xml')
body_cascade = cv2.CascadeClassifier(r'C:\Users\HP\Downloads\haarcascade_fullbody.xml')

age_net = cv2.dnn.readNetFromCaffe(r'C:\Users\HP\Downloads\deploy_age.prototxt', r'C:\Users\HP\Downloads\age_net.caffemodel')

AGE_LIST = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']

# Create main application window with a modern theme
root = ThemedTk(theme="plastik")
root.title("Face, Body, and Age Detection")
root.geometry("900x650")

# Set a gradient background color or a custom image
bg_color1 = "#87CEFA"
bg_color2 = "#4682B4"
canvas = tk.Canvas(root, width=900, height=650)
canvas.pack(fill="both", expand=True)

# Gradient background function
def create_gradient(canvas, color1, color2):
    for i in range(650):
        r = int(int(color1[1:3], 16) + (int(color2[1:3], 16) - int(color1[1:3], 16)) * i / 650)
        g = int(int(color1[3:5], 16) + (int(color2[3:5], 16) - int(color1[3:5], 16)) * i / 650)
        b = int(int(color1[5:7], 16) + (int(color2[5:7], 16) - int(color1[5:7], 16)) * i / 650)
        color = f'#{r:02x}{g:02x}{b:02x}'
        canvas.create_line(0, i, 900, i, fill=color)

create_gradient(canvas, bg_color1, bg_color2)

# Frame for holding the video feed and detection results
video_frame = Frame(root, bg="white", relief="raised", borderwidth=10)
video_frame.place(relx=0.5, rely=0.4, anchor="center", width=640, height=480)

# Video label to display feed
video_label = Label(video_frame, bg="black")
video_label.pack()

# Variables to store detection results
age_label = tk.StringVar()
age_label.set("Age: N/A")
detected_label = tk.StringVar()
detected_label.set("Detection: None")

# Create labels for detection info with custom fonts and styling
age_text = Label(root, textvariable=age_label, font=("Helvetica", 18, "bold"), fg="white", bg=bg_color2)
age_text.place(relx=0.5, rely=0.7, anchor="center")
detection_text = Label(root, textvariable=detected_label, font=("Helvetica", 18, "bold"), fg="white", bg=bg_color2)
detection_text.place(relx=0.5, rely=0.75, anchor="center")

# Capture video
cap = cv2.VideoCapture(0)

# Function to perform detection
def detect():
    ret, frame = cap.read()
    if not ret:
        return

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces and bodies
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    bodies = body_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]
        blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
        age_net.setInput(blob)
        age_preds = age_net.forward()
        age = AGE_LIST[age_preds[0].argmax()]

        # Draw rectangle and update age label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        age_label.set(f"Age: {age}")
        detected_label.set("Detection: Face Detected")

    for (x, y, w, h) in bodies:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        detected_label.set("Detection: Body Detected")

    # Convert frame to ImageTk format for display
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = Image.fromarray(img)
    imgtk = ImageTk.PhotoImage(image=img)

    # Update video label with frame
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    # Run detection in loop
    video_label.after(10, detect)

# Button styling for start and stop
button_style = {
    "font": ("Helvetica", 16, "bold"),
    "bg": "#FF4500",
    "fg": "white",
    "activebackground": "#DC143C",
    "activeforeground": "white",
    "relief": "raised",
    "bd": 5,
    "cursor": "hand2",
}

# Start button to begin detection
start_btn = Button(root, text="Start Detection", command=detect, **button_style)
start_btn.place(relx=0.4, rely=0.85, anchor="center", width=200, height=50)

# Function to stop the detection and close the window
def stop_detection():
    cap.release()
    root.quit()

# Stop button to end detection
stop_btn = Button(root, text="Stop Detection", command=stop_detection, **button_style, bg="#4682B4")
stop_btn.place(relx=0.6, rely=0.85, anchor="center", width=200, height=50)

# Run the application
root.mainloop()
